# Sesiones prácticas

## 0

Instalación de Python + ecosistema científico + opencv + otros paquetes ([install.ipynb](install.ipynb)).

- prueba de scripts
- manejo básico de jupyter
- repaso Python/numpy

## 1

a) Tutorial del acceso a diferentes fuentes de imágenes con la utilidad `autoStream` de umucv ([captura.ipynb](captura.ipynb)).

b) Calibración de la cámara (scripts en `code/calibrate`).

c) Script de acceso a los *hand landmarks* de mediapipe (`code/DL/mp_hands/hands.py`).

## 2

Ejemplos de código y utilidades.

- `webcam.py` con opencv crudo vs `stream.py`
- selección de ROIS (cv.selectROI vs umucv.util.ROI)
- Recortar y unir imágenes para conseguir [algo como esto](../images/demos/ej-c0.png).
- control de webcam v4l2-ctl, vlc, gucview (`util/focus.sh`, `util/power.sh`)
- `util/wzoom.py` (para las ventanas de Windows que no tienen zoom)
- `color/histogram.py`, `color/histogram2.py`

Medidor de ángulos:
- captura de coordenadas: `util/mouse.py`
- círculos en las posiciones marcadas (cv.circle)
- coordenadas textuales (cv.putText  (ej. en `hello.py`) o umucv.util.putText)
- marcar solo los dos últimos (pista: collections.deque)
- reproducir code/medidor.py indicando la distancia en pixels
- Dado el FOV: indicar el ángulo de las direcciones marcadas.

## 3

Ejemplos:
- `util/trackbar.py`, `color/inrange0.py`, `color/inrange.py`
- `util/help_window.py`
- `util/save_video.py`

Filtros:
- Ejemplos de filtros (media, gaussiano, mediana, etc.)
- Comparación de la calidad de suavizado y del tiempo de cálculo de un Box Filter vs un filtro Gaussiano (`box_vs_gaussian.py`).
- Ejercicio de detección de zonas interesantes en la imagen mediante una máscara obtenida como umbralización del promedio local de las diferencias entre cada pixel original y su valor suavizado (`boxy.py`).

Ejercicio: subir a la carpeta personal del aula virtual un vídeo muy corto (5s) demostrando el funcionamiento del script realizado en clase.

## 4

Ejemplos:

- Argumentos en la línea de órdenes coexistiendo con los de autoStream (`util/args.py`).
- Detección de actividad: Selección de región y cómputo de la suma de diferencias absolutas respecto al recorte guardado.
- Efecto "fantasmal" (`util/deque.py`). 
- Reproducirlo actualizando el valor medio con coeficientes $\alpha$ y $1-alpha$ en vez de operar con n imágenes. Crear modelo de fondo con media y desviación típica.
- Sustracción de fondo (`color/backsub0.py`, `color/backsub.py`).
- Probarlo en vídeo de péndulo, stream del puente, etc.
- Segmentador de mediapipe.
- rembg.

Ejercicio: subir a la carpeta personal una imagen que muestre un caso de fallo de `code/DL/mp_segmenter0.py` y opcionalmente comparar con el resultado que consigue `code/DL/rembg/test_rembg.py`.

## 5

- Ampliación de `roi.py` para captura de múltiples regiones. Mostrarlas en una ventana seleccionándola con teclas, o todas juntas reescaladas a un número de filas común, usando np.hstack.
- clasificador basado en `DL/mp_embedder/similarity.py`
- Gestos con distancia procrustes
- Componentes conexas, extracción de contornos (`code/inrange.py`).
- Explicación del reconocimiento frecuencial de siluetas.
- Ejemplo paso a paso en `code/shape/trebol*.py`.

## 6

- `code/hog/hog0.py`
- `code/hog/pedestrian.py`
- `code/hog/facelandmarks.py` y efectos especiales.
- Captura asíncrona `code/thread`

## 7

- Repaso de *image matching* mediante correlación cruzada (`code/crosscorr.py`)
- Demostraciones de detección de movimiento de cámara y de flujo óptico denso.
- Detección de *corners* y tracker de Lucas-Kanade (`code/LK`)

## 8

- Detector SIFT de OpenCV (`code/SIFT`)
- Prueba de concepto de detección de escala con el operador Hessiano (`code/SIFT`).
- Uso de nuestros prototipos de visión con el teléfono móvil mediante bots de telegram (`code/bot`).

## 9

- Repaso de machine learning
- Clasificador de dígitos manuscritos sobre secuencias de imágenes naturales con PCA + modelos gaussianos (`code/DL/CNN`)
- Mejora utilizando una red convolucional

## 10

Actualización del paquete umucv: En la raíz del repositorio ejecutamos `pip install package/`

- Ejemplos de uso de modelos de deep learning interesantes y sus variantes disponibles en mediapipe (`code/DL`).

## 11

- Demostración de puntos de fuga con coordenadas homogéneas.
- Repaso del notebook de transformaciónes del plano (`transf2D.ipynb`).
- Ejemplos de rectificación de planos y otras transformaciones relacionadas ("stitching", sustitución de cuadrángulos, etc.).
- Detección y rectificación de marcador (`code/polygons`).

## 12

- Ejemplos de realidad aumentada con marcador artificial (`code/pose`)
- Entrenamiento de una U-NET desde cero en un problema muy sencillo (`code/DL/UNET`).

## 13

- Ejemplos de reconstruccion 3D
- Repaso y dudas